---

* 출처: LangChain 공식 문서 또는 해당 교재명
* 원본 URL: https://smith.langchain.com/hub/teddynote/summary-stuff-documents

---

## **직렬화(`Serialization`)**

### **`직렬화`란?**

#### **정의**

* 모델을 `저장 가능한 형식`으로 `변환`하는 과정

* 모델 직렬화
  * `AI 개발` 및 `배포 과정`에서 `중요한 단계`
  * `효율적인 모델 관리`와 `재사용`을 가능하게 함

#### **목적**

* 모델 **재사용** (재훈련 없이)
* 모델 **배포 및 공유 용이**
* **계산 리소스 절약**

#### **장점**

* **`빠른` 모델 로딩**
* **`버전 관리` 가능**
* **`다양한 환경`에서 사용 가능**

---

* `is_lc_serializable 클래스 메서드` → **`LangChain` 클래스가 직렬화 가능한지 확인**

In [1]:
# 환경변수 처리 및 클라어트 생성
from langsmith import Client
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
import os
import json

# 클라이언트 생성 
api_key = os.getenv("LANGSMITH_API_KEY")
client = Client(api_key=api_key)

In [ ]:
# LangSmith 추적 설정하기 (https:smith.langchin.com)
# LangSmith 추적을 위한 라이브러리 임포트
from langsmith import traceable                                                             # @traceable 데코레이터 사용 시

# LangSmith 환경 변수 확인

print("\n--- LangSmith 환경 변수 확인 ---")
langchain_tracing_v2 = os.getenv('LANGCHAIN_TRACING_V2')
langchain_project = os.getenv('LANGCHAIN_PROJECT')
langchain_api_key_status = "설정됨" if os.getenv('LANGCHAIN_API_KEY') else "설정되지 않음"      # API 키 값은 직접 출력하지 않음
org = "설정됨" if os.getenv('LANGCHAIN_ORGANIZATION') else "설정되지 않음"                     # 직접 출력하지 않음

if langchain_tracing_v2 == "true" and os.getenv('LANGCHAIN_API_KEY') and langchain_project:
    print(f"✅ LangSmith 추적 활성화됨 (LANGCHAIN_TRACING_V2='{langchain_tracing_v2}')")
    print(f"✅ LangSmith 프로젝트: '{langchain_project}'")
    print(f"✅ LangSmith API Key: {langchain_api_key_status}")
    print("  -> 이제 LangSmith 대시보드에서 이 프로젝트를 확인해 보세요.")
else:
    print("❌ LangSmith 추적이 완전히 활성화되지 않았습니다. 다음을 확인하세요:")
    if langchain_tracing_v2 != "true":
        print(f"  - LANGCHAIN_TRACING_V2가 'true'로 설정되어 있지 않습니다 (현재: '{langchain_tracing_v2}').")
    if not os.getenv('LANGCHAIN_API_KEY'):
        print("  - LANGCHAIN_API_KEY가 설정되어 있지 않습니다.")
    if not langchain_project:
        print("  - LANGCHAIN_PROJECT가 설정되어 있지 않습니다.")

<small>

* 셀 출력

    ```markdown
    --- LangSmith 환경 변수 확인 ---
    ✅ LangSmith 추적 활성화됨 (LANGCHAIN_TRACING_V2='true')
    ✅ LangSmith 프로젝트: 'LangChain-prantice'
    ✅ LangSmith API Key: 설정됨
    -> 이제 LangSmith 대시보드에서 이 프로젝트를 확인해 보세요.
    ```

* **`모델` 생성**

In [3]:
# LLM 초기화
from langchain_google_genai import ChatGoogleGenerativeAI

gemini_lc = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash-lite",
        #temperature=0.7,                                    
        max_output_tokens=4096,
    )

* **`프롬프트` 생성**

In [4]:
# 프롬프트 생성

prompt = PromptTemplate.from_template("{fruit}의 색상이 무엇입니까?")

---

* **`클래스(class)` 직렬화 가능 여부 확인하기**

In [ ]:
# 직렬화 가능한지 체크하기

print(f"ChatGoogleGenerativeAI: {ChatGoogleGenerativeAI.is_lc_serializable()}")

<small>

* 셀 출력

    ```markdown
    ChatGoogleGenerativeAI: True
    ```

* **`LLM 객체` 직렬화 가능 여부 확인하기**

In [ ]:
# 교재와 동일하게 온도를 0으로 맞춰 초기화하기

gemini_lc = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash-lite",
        temperature=0,                          # 교재와 동일한 조건        
        max_output_tokens=4096,
    )

# 직렬화 가능 여부 체크하기
print(f"ChatGoogleGenerativeAI:{gemini_lc.is_lc_serializable()}")

<small>

* 셀 출력

    ```markdown
    ChatGoogleGenerativeAI:True
    ```

---

* **`체인` 생성**

In [ ]:
# 체인 생성
chain = prompt | gemini_lc

# 직렬화 가능 여부 체크하기
chain.is_lc_serializable()                      # True

---

### **체인(`Chain`) 직렬화(`dumps`, `dumpd`)**

#### **개요**

* 체인 직렬화: `직렬화 가능한 모든 객체`를 **`딕셔너리`** 또는 **`JSON 문자열`** 로 변환하는 과정을 의미

#### **직렬화 방법**

* 객체의 속성 및 데이터를 `키-값 쌍`으로 저장하여 `딕셔너리 형태`로 변환

  * 객체를 `쉽게 저장`하고 `전송`할 수 있게 함
  * `다양한 환경`에서 `객체`를 `재구성 가능`

<br>

* 참고

  * **`dumps`**: 객체를 `JSON 문자열`로 직렬화
  * **`dumpd`**: 객체를 `딕셔너리`로 직렬화

#### **1) `dumpd` 사용해서 직렬화하기**

In [ ]:
from langchain_core.load import dumpd, dumps

dumpd_chain = dumpd(chain)
dumpd_chain

<small>

* 셀 출력

    ```python
    {'lc': 1,
    'type': 'constructor',
    'id': ['langchain', 'schema', 'runnable', 'RunnableSequence'],
    'kwargs': {'first': {'lc': 1,
    'type': 'constructor',
    'id': ['langchain', 'prompts', 'prompt', 'PromptTemplate'],
    'kwargs': {'input_variables': ['fruit'],
        'template': '{fruit}의 색상이 무엇입니까?',
        'template_format': 'f-string'},
    'name': 'PromptTemplate'},
    'last': {'lc': 1,
    'type': 'constructor',
    'id': ['langchain_google_genai', 'chat_models', 'ChatGoogleGenerativeAI'],
    'kwargs': {'model': 'models/gemini-2.5-flash-lite',
        'google_api_key': {'lc': 1, 'type': 'secret', 'id': ['GOOGLE_API_KEY']},
        'temperature': 0.0,
        'max_output_tokens': 4096,
        'n': 1,
        'max_retries': 6,
        'default_metadata': []},
    'name': 'ChatGoogleGenerativeAI'}},
    'name': 'RunnableSequence'}
    111

In [ ]:
# 직렬화된 체인 타입 확인하기

type(dumpd_chain)                       # dict

#### **2) `dumps` 사용해서 직렬화하기**

In [ ]:
# dumps 함수를 사용 → 직렬화된 체인 확인해보기

dumps_chain = dumps(chain)

dumps_chain

<small>

* 셀 출력

    ```json
    '{"lc": 1, "type": "constructor", "id": ["langchain", "schema", "runnable", "RunnableSequence"], "kwargs": {"first": {"lc": 1, "type": "constructor", "id": ["langchain", "prompts", "prompt", "PromptTemplate"], "kwargs": {"input_variables": ["fruit"], "template": "{fruit}\\uc758 \\uc0c9\\uc0c1\\uc774 \\ubb34\\uc5c7\\uc785\\ub2c8\\uae4c?", "template_format": "f-string"}, "name": "PromptTemplate"}, "last": {"lc": 1, "type": "constructor", "id": ["langchain_google_genai", "chat_models", "ChatGoogleGenerativeAI"], "kwargs": {"model": "models/gemini-2.5-flash-lite", "google_api_key": {"lc": 1, "type": "secret", "id": ["GOOGLE_API_KEY"]}, "temperature": 0.0, "max_output_tokens": 4096, "n": 1, "max_retries": 6, "default_metadata": []}, "name": "ChatGoogleGenerativeAI"}}, "name": "RunnableSequence"}'
    ```

In [ ]:
# 직렬화된 체인의 타입 확인하기

type(dumps_chain)                           # str

---

### **`Pickle` 파일**

#### **개요**

* `Pickle` 파일 = `Python 객체`를 `바이너리 형태`로 `직렬화`하는 `포맷`

#### **특징**

* **개요**
  * Python 객체를 바이너리 형태로 직렬화하는 포맷
  * `효율적인 모델 관리`와 `재사용`을 가능하게 함

    ---

* 특징

  * **`Python 전용`** (다른 언어와 호환 불가)
  * 대부분의 `Python 데이터 타입` 지원 (`리스트`, `딕셔너리`, `클래스` 등)
  * `객체의 상태`와 `구조`를 그대로 보존

    ---

* 장점

  * `효율적`인 `저장` 및 `전송`
  * `복잡한 객체 구조 유지`
  * **빠른 `직렬화/역직렬화` 속도**

    ---

* 단점

  * **보안 위험** (`신뢰할 수 없는 데이터 역직렬화 시 주의 필요`)
  * **사람이 읽을 수 없는 `바이너리 형식`**

#### **주요 용도**

* 객체 `캐싱`
* 머신러닝 `모델 저장`
* 프로그램 `상태 저장` 및 `복원`

#### **사용법**

* **`pickle.dump()`**: 객체를 `파일`에 `저장`

* **`pickle.load()`**: `파일`에서 `객체` `로드`

#### **1) `Pickle` 파일로 저장하기**

* `Pickle` 파일로 저장하기

In [ ]:
import pickle

# fuit_chain.pkl 파일로 직렬화된 체인 저장하기
with open("fruit_chain.pkl", "wb") as f:
    pickle.dump(dumpd_chain, f)                             # 04_Model/fruit_chain.pkl 생성됨

* `JSON` 형식으로도 저장해보기

In [ ]:
import json

with open("fruit_chain.json", "w") as fp:
    json.dump(dumpd_chain, fp)
                                                            # 04_Model/fruit_chain.json 셍성됨

#### **2) `load`: 저장한 모델 불러오기**

* 먼저, 이전에 저장한 `pickle` 형식 파일 로드

In [ ]:
import pickle

# pickle 파일 로드
with open("fruit_chain.pkl", "rb") as f:
    loaded_chain = pickle.load(f)                           

* 로드한 `JSON`파일을 `load`메서드로 로드하기

In [ ]:
from langchain_core.load import load

# 체인 로드하기
chain_from_file = load(loaded_chain)

# 체인 실행하기
print(chain_from_file.invoke({"fruit": "사과"})) 

<small>

* 셀 출력 (1.1s)

    ```markdown
    /var/folders/h3/l7wnkv352kqftv0t8ctl2ld40000gn/T/ipykernel_3877/1712892737.py:4: LangChainBetaWarning: The function `load` is in beta. It is actively being worked on, so the API may change.
    ```

    ```python
    chain_from_file = load(loaded_chain)
    ```

    ```python
    content='사과는 일반적으로 **빨간색**입니다.\n\n하지만 사과의 품종에 따라 **초록색**, **노란색**, 또는 이 색들이 섞인 **붉은색과 초록색이 섞인 색** 등 다양한 색깔을 띨 수 있습니다.' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': []} id='run--5a09635a-b028-4167-abbb-831462ada1f1-0' usage_metadata={'input_tokens': 9, 'output_tokens': 61, 'total_tokens': 70, 'input_token_details': {'cache_read': 0}}
    ```

<br>

  ---

<br>

* 셀 출력 해석하기

  * ➀ 경고 메시지
  
    * `a. 경고 메시지_1`
  
      ```markdown
      /var/folders/h3/l7wnkv352kqftv0t8ctl2ld40000gn/T/ipykernel_3877/1712892737.py:4: LangChainBetaWarning: 
      ```
    
    * `LangChainBetaWarning`은 **현재 사용 중인 load 함수가 베타 버전이라, API(인터페이스)가 변경될 수 있다** 는 의미
    * 즉, 이 기능은 아직 개발 중이라는 뜻 → 그래서 나중에 바뀔 수도 있다는 경고 출력한 것임

    * `b. 경고 메시지_2`
  
      ```markdown
        The function `load` is in beta. It is actively being worked on, so the API may change.
      ```

    * 아까 설명한 경고 메시지의 자세한 내용
    * **현재 load 함수는 베타(시험) 버전이기 때문에, 계속해서 작업이 이루어지고 있고, 나중에 함수 사용법이나 결과가 바뀔 수 있다**

    <br>

    ---

    <br>

  * ➁ `load` 함수 호출 = `loaded_chain 객체 로드
  
    ```python
    chain_from_file = load(loaded_chain)
    ```

    * 실제로 코드에서 `load 함수`를 호출해서 `loaded_chain 객체`를 `로드`하는 부분
    * `chain_from_file` 변수 = `load 함수가 반환하는 값`을 `저장`하는 역할

    <br>

    ---

    <br>

  * ➂ `content` 내용
  
    * `a. chain_from_file.invoke()를 실행했을 때 얻은 결과`

      ```python
      content='사과는 일반적으로 **빨간색**입니다.\n\n하지만 사과의 품종에 따라 **초록색**, **노란색**, 또는 이 색들이 섞인 **붉은색과 초록색이 섞인 색** 등 다양한 색깔을 띨 수 있습니다.'
      ```

      * `content` = `invoke 함수`가 **입력한 데이터** (여기서는 `"fruit": "사과"`)에 대해 **답을 만들어낸 결과** 에 해당
      * 사과에 대한 설명
        * 사과의 `색깔`에 대해 설명하고 있음 
        * **`사과는 일반적으로 빨간색이지만, 품종에 따라 초록색, 노란색, 혹은 빨간색과 초록색이 섞인 색 등이 있다`**

    * `b. 추가적인 매개변수 여부`

      ```python
      additional_kwargs={}
      ```
      
      * `additional_kwargs` = **비어있음** = **추가적인 매개변수(인자들)가 없음**
      * 특별히 추가된 옵션 없이 `기본적인 결과만 받았음`

    * `c. 응답 데이터 부분`

      ```python
      response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': []}
      ```
      
      * `finish_reason: 'STOP'` = **모델이 답을 완료함** = 모델이 더 이상 응답할 필요가 없다고 판단
      * `model_name: 'gemini-2.5-flash-lite'` = **사용된 모델 이름**
      * `safety_ratings: []` = **비어있음** = 안전성 평가 결과가 없음 = `안전성에 관한 별도의 평가가 없다는 의미`
      * `prompt_feedback: {'block_reason': 0, 'safety_ratings': []}` = **프롬프트(질문)에 대한 피드백**
        * `block_reason: 0` = **아무 문제 없이 정상적으로 처리되었음**

    * `d. 실행된 모델의 고유 ID`

      ```python
      id='run--5a09635a-b028-4167-abbb-831462ada1f1-0'
      ```
      
      * **모델이 실행된 고유한 ID**
      * 여러 번 실행했을 때, 각 실행을 `구별할 수 있도록` 도와줌

    * `e. 사용량 메타데이터` 

      ```python
      usage_metadata={'input_tokens': 9, 'output_tokens': 61, 'total_tokens': 70, 'input_token_details': {'cache_read': 0}}
      ```
      
      * `input_tokens: 9` = **입력한 데이터(여기서는 `"fruit": "사과"`)에 사용된 토큰의 수**
      * `output_tokens: 61` = **모델이 생성한 답변에서 사용된 토큰의 수**
      * `total_tokens: 70` = **입력과 출력 토큰을 합친 총 토큰 수** = `9+61`
      * `input_token_details: {'cache_read': 0}` = **캐시에서 읽어온 데이터가 없음**
        * 즉, 이 데이터는 `이전에 저장된 데이터를 사용하지 않고` **`새로 계산한 결과라는 의미`**

---

* **`Q. load 함수가 불안정하다면` 대신 사용할 수 있는 방법은?**

  * `langchain` 라이브러이에서 제공하는 다른 방법 사용하기
  * `langchain` 라이브러리 최신 버전으로 업데이트하며 사용하기

    ```python

       # bash
       pip install --upgrade langchain
    
    ```

  * 업데이트 후 테스트 후 사용 or **에외 처리** 추가하여 사용하기

    ```python

        # 예외처리 예시

        try:
            chain_from_file = load(loaded_chain)
            result = chain_from_file.invoke({"fruit": "사과"})

        except Exception as e:
            print("오류 발생:", e)
            # 오류 발생 시 다른 방식으로 처리
            result = "기본 결과"
            
        print(result)

    ```

---

In [ ]:
from langchain_core.load import load, loads

load_chain = load(
    loaded_chain, secrets_map={"GOOGLE_API_KEY": os.environ["GOOGLE_API_KEY"]}
)

# 불러온 체인이 정상 동작하는지 확인하기
load_chain.invoke({"fruit": "사과"})

<small>

* 셀 출력 (1.0s)

    ```python
    AIMessage(content='사과는 일반적으로 **빨간색**입니다.\n\n하지만 사과의 품종에 따라 **초록색**, **노란색**, 또는 이 색들이 섞인 **붉은색과 초록색이 섞인 색** 등 다양한 색깔을 띨 수 있습니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': []}, id='run--2b727704-2ef6-4351-bbb1-9ab22e4c3247-0', usage_metadata={'input_tokens': 9, 'output_tokens': 61, 'total_tokens': 70, 'input_token_details': {'cache_read': 0}})
    ```

In [ ]:
# json 파일로드
with open("fruit_chain.json", "r") as fp:
    loaded_from_json_chain = json.load(fp)
    loads_chain = load(loaded_from_json_chain)
    
# 불러온 체인이 정상 동작하는지 확인하기
loads_chain.invoke({"fruit": "사과"})

<small>

* 셀 출력 (0.9s)

    ```python
    AIMessage(content='사과는 일반적으로 **빨간색**입니다.\n\n하지만 사과의 품종에 따라 **초록색**, **노란색**, 또는 이 색들이 섞인 **붉은색과 초록색이 섞인 색** 등 다양한 색깔을 띨 수 있습니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': []}, id='run--d27c9e1c-b3f1-4561-b844-9425bcb6824e-0', usage_metadata={'input_tokens': 9, 'output_tokens': 61, 'total_tokens': 70, 'input_token_details': {'cache_read': 0}})
    ```

---

* *next: 토큰 사용량 확인*

---